In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import glob

In [ ]:
heart_df = pd.read_csv('../input/heart-attack-analysis-prediction-dataset/heart.csv')
heart_df.head()

In [ ]:
heart_df.tail()

* categorical data : 'sex' , 'cp' , 'fbs' , 'reestecg', 'exng' , 'slp'
* Numerical data: 'age' , 'trtbps' , 'chol' , 'thalachh' , 'oldpeak'

In [ ]:
o2sat_df = pd.read_csv('../input/heart-attack-analysis-prediction-dataset/o2Saturation.csv')
o2sat_df.head()

# Data analysis

In [ ]:
heart_df.info()

In [ ]:
heart_df.isna().sum()

There is no missing values

In [ ]:
heart_df.columns

In [ ]:
heart_df.describe().round(1)

In [ ]:
X = heart_df.drop('output',axis=1)
y = heart_df['output']

In [ ]:
plt.figure(figsize=(10,10))
corr = heart_df.corr()
sns.heatmap(corr,annot= True, fmt='2.00%')
plt.show()

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,classification_report
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)
print(len(X_train))

In [ ]:
scaling = MinMaxScaler(feature_range=(0,1))
X_train = scaling.fit_transform(X_train)
X_test = scaling.transform(X_test)

In [ ]:
model_rfc = RandomForestClassifier(random_state=0).fit(X_train,y_train)
predict_rfc = model_rfc.predict(X_test)
predict_proba = model_rfc.predict_proba(X_test)[:,1]

In [ ]:
predict_proba.shape

In [ ]:
print(classification_report(y_test,predict_rfc))

In [ ]:
def true_false_positive(threshold_vector, y_test):
    true_positive = np.equal(threshold_vector, 1) & np.equal(y_test, 1)
    true_negative = np.equal(threshold_vector, 0) & np.equal(y_test, 0)
    false_positive = np.equal(threshold_vector, 1) & np.equal(y_test, 0)
    false_negative = np.equal(threshold_vector, 0) & np.equal(y_test, 1)

    tpr = true_positive.sum() / (true_positive.sum() + false_negative.sum())
    fpr = false_positive.sum() / (false_positive.sum() + true_negative.sum())

    return tpr, fpr

In [ ]:
def roc_from_scratch(probabilities, y_test, partitions=100):
    roc = np.array([])
    for i in range(partitions + 1):
        
        threshold_vector = np.greater_equal(probabilities, i / partitions).astype(int)
        tpr, fpr = true_false_positive(threshold_vector, y_test)
        roc = np.append(roc, [fpr, tpr])
        
    return roc.reshape(-1, 2)

In [ ]:
ROC = roc_from_scratch(predict_proba,y_test)
plt.plot(ROC[:,0],ROC[:,1],color='#0F9D58')
plt.title('ROC Curve',fontsize=20)
plt.xlabel('False Positive Rate',fontsize=16)
plt.ylabel('True Positive Rate',fontsize=16)
plt.show()

### Note I need to test another model like SVC, KNeighborsClassifier